In [1]:
import json
import re

In [2]:
questions = dict()
cal_reasoning = 0
with open('../data/qwen2.5-coder-3b-sft__hard_v0_4.jsonl') as f:
    for line in f:
        data = json.loads(line)
        ids = data['ids']
        questions[ids] = data
        if 'reasoning' in data:
            cal_reasoning += 1
cal_reasoning

1915

In [3]:
len(questions)

1915

In [4]:
# with open('../data/Qwen__Qwen2.5-32B-Instruct-GPTQ-Int4-scored-gpt-4o-mini__v5_good.jsonl') as f:
#     for line in f:
#         data = json.loads(line)
#         ids = data['ids']
#         if 'score' not in questions[ids]:
#             questions[ids]['score'] = []
#         questions[ids]['score'].append(data['score'])

with open('../data/gemini-2.0-flash-scored-qwen2.5-coder-3b-sft__hard_v0_4.jsonl') as f:
    score = 0
    for line in f:
        data = json.loads(line)
        ids = data['ids']
        if 'score' not in questions[ids]:
            questions[ids]['score'] = []
        questions[ids]['score'].append(data['score'])
        score += data['score']
    print(score)

with open('../data/gpt-4o-mini-scored-qwen2.5-coder-3b-sft__hard_v0_4.jsonl') as f:
    score = 0
    for line in f:
        data = json.loads(line)
        ids = data['ids']
        if 'score' not in questions[ids]:
            questions[ids]['score'] = []
        questions[ids]['score'].append(data['score'])
        score += data['score']
    print(score)

1239
981


In [5]:
len_sql = 0
for ids, data in questions.items():
    if 'score' in data:
        data['avg'] = sum(data['score']) / 2 # len(data['score'])
    # len_sql += len(data['sql'])
len_sql /= len(questions)
len_sql


0.0

In [6]:
good_ids = set()
for ids, data in questions.items():
    if 'score' in data and data['avg'] == 1:
        good_ids.add(ids)
        # if len(data['sql']) > 1:
        #     print(ids, data)

In [7]:
len(good_ids)

917

In [38]:
for ids, data in questions.items():
    if ids in good_ids:
        print(ids, data)
        break

4429b8ac-ce3d-43e6-b8e3-13e7e47f37b5 {'date': '2025-08-02 23:54:01', 'ids': '4429b8ac-ce3d-43e6-b8e3-13e7e47f37b5', 'message': [{'role': 'system', 'content': "You are an expert in financial statement and database management. You will be asked to convert a natural language query into a SQL query. \nYou will have the following database description:\n\n### Database Description\nYou're given database about the financial statments of top Vietnamese companies, either bank, cooperates and securities.\n\n<overall_description>\n\nAll the data in the financial statments are followed by the regulation of Vietnamese Accounting Standard (VAS). The English translations are followed by the International Financial Reporting Standards (IFRS).\n\nThere are 3 type of financial statments, based on VAS regulation: bank, non-bank corporation and securities firm (firms that provide stock options and financial instruments).\nAll 3 type of reports are stored in one single table, and there will be a sight diffe

In [8]:
output_path = '../data/qwen2.5-coder-3b-sft__hard_v0_4_good.jsonl'
with open(output_path, 'w') as f:
    for ids, data in questions.items():
        if ids in good_ids:
            f.write(json.dumps(data) + '\n')

In [39]:
with open('../data/message-gpt-4o-mini__v5_2_good.jsonl', 'w') as f:
    for ids, data in questions.items():
        if ids in good_ids:
            
            f.write(json.dumps(data['message']) + '\n')

In [13]:
count_result_format = 0
good_result_format = 0
for ids, data in questions.items():
    if 'result format' in data['reasoning'].lower():
        count_result_format += 1
        if '### Result Format' in data['reasoning']:
            good_result_format += 1
            print(ids, data['reasoning'])
            break
count_result_format, good_result_format

(0, 0)

In [ ]:
for ids, data in questions.items():
  if ids == '74710a9c-1b03-4821-b306-61057c8200cf':
    print('QUESTION ===========')
    print(data['question'])  
    print('TABLES ===========')
    print(data['table'])
    print('ANSWERS ===========')
    print(data['reasoning'])
    print('AVG ===========')
    print(data['avg'])
    break

In [1]:
import random
import json

In [2]:
random.randint(0, 1)

1

In [13]:
import tiktoken

def count_tokens(text: str, model: str = "gpt-4") -> int:
    # Select the tokenizer for the model
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

err = 0
total_tokens = 0
num = 1
s = 0
u = 0
a = 0
with open('../data/message_ner_gemini-2.0-flash__v5.jsonl') as f:
    for line in f:
        try:
            data = json.loads(line)

            # with open('../data/message_gpt-4o-mini__v4_good_good.jsonl', 'a') as f:
            #     json.dump(data, f)
            #     f.write('\n')

            for message in data:
                text = message['content']
                total_tokens += count_tokens(text)
                if message['role'] == 'system':
                    s += count_tokens(text)
                elif message['role'] == 'user':
                    u += count_tokens(text)
                else:
                    a += count_tokens(text)
        except Exception as e:
            err += 1
        num += 1
total_tokens, err, u, s, a

(420490, 0, 307367, 0, 113123)

In [6]:
with open('../data/message_gpt-4o-mini__v3_good.jsonl') as f:
    for line in f:
        try:
            data = json.loads(line)
            new_data = []
            for message in data:
                role = message['role']
                content = message['content'].strip()
                new_data.append({'role': role, 'content': content})
            with open('../data/message_gpt-4o-mini__v3_good_good.jsonl', 'a') as f:
                json.dump(new_data, f)
                f.write('\n')
                

        except Exception as e:
            print(e)

## Grade Cell

In [3]:
import json

evaluate_path = '../data/gemini-2.0-flash-evaluate-qwen2.5-coder-3b-kto__sql_v0.jsonl'
mcq_path = '../data/mcq_v0.jsonl'

evaluate = dict()
cant_ansewer = 0
score = 0

total_questions = 0
with open(mcq_path) as f:
    for line in f:
        data = json.loads(line)
        total_questions += 1

return_question = 0
with open(evaluate_path) as f:
    for line in f:
        data = json.loads(line)
        evaluate[data['ids']] = data
        if 'score' in data:
            score += data['score']
        if 'cant_answer' in data and data['cant_answer']:
            cant_ansewer += 1
        return_question += 1

total_questions, return_question, cant_ansewer, score, score/len(evaluate), score/total_questions

(423, 423, 91, 299, 0.706855791962175, 0.706855791962175)

### Qwen 2.5 Coder 3B
SFT 3B: 0.7115

DPO 3B: 0.6761

KTO 3B: 0.7069

Original: 0.2553

### Qwen 2.5 Coder 1.5B
SFT 1.5B: 0.6525

DPO 1.5B (failed) 1.5B: 0.6288

KTO 1.5B: 0.6643 (Loss 13 questions during saving process. 0.6835 for 410 ques) 

SFT Extended 1.5B: 


### Other
GPT 4o mini: 0.64775

## Check bad Generation

In [15]:
sql_path = '../data/deepseek-chat__v0_good.jsonl'
new_sql_path = '../data/sql_v0.jsonl'

with open(sql_path) as f:
    for line in f:
        data = json.loads(line)
        ids = data['ids']
        if evaluate.get(ids, dict()).get('score', 0) == 1:
            with open(new_sql_path, 'a') as f:
                json.dump(data, f)
                f.write('\n')

In [14]:
new_mcq_path = '../data/mcq_v0.jsonl'
with open(mcq_path) as f:
    for line in f:
        data = json.loads(line)
        ids = data['ids']
        if evaluate.get(ids, dict()).get('score', 0) == 1:
            with open(new_mcq_path, 'a') as f:
                json.dump(data, f)
                f.write('\n')


In [8]:
def is_npl(question: dict):
    if 'non-per' in question['question'].lower() and 'bdr' not in question['reasoning'].lower():
        return True
    return False


def contain_en_company(question: dict):
    if 'yoy' in question['reasoning'].lower() and 'quarter = 0' not in question['reasoning'].lower():
        return True
    return False

def contain_data(question, data):
    if data in question['reasoning'].lower():
        return True
    return False

In [11]:
file_path = '../data/gpt-4o-mini__v5.jsonl'
new_file_path = '../data/gpt-4o-mini__v5_2.jsonl'

In [ ]:
import json
err = 0

err_id = ''


with open(file_path) as f:
    for line in f:
        data = json.loads(line)
        if contain_data(data, 'average'):
            err += 1
            err_id = data['ids']
        else:
            print('======== QUESTION ===========')
            print(data['question'])  
            print('======== TABLES ===========')
            print(data['table'])
            print('======== ANSWERS ===========')
            print(data['reasoning'])
            # print('AVG ===========')
            # print(data['avg'])
        #     break
        # else:
        #     with open(new_file_path, 'a') as f:
        #         json.dump(data, f)
        #         f.write('\n')

In [15]:
err

17

In [14]:
err_id

'69ecf8be-f65c-484c-87d6-069ff8fa530d'

## BATCH LLM

In [89]:
import sys 
sys.path.append('../')
from llm import ChatGPT

In [90]:
llm = ChatGPT()

In [91]:
import json

In [92]:
msg_path = '../data/template_gemini-2.0-flash-lite-preview-02-05__v5.jsonl'
questions = []
messages = []
with open(msg_path) as f:
    for line in f:
        try:
            data = json.loads(line)
            questions.append(data)
            messages.append(data['message'])
        except Exception as e:
            print(e)

Expecting ',' delimiter: line 1 column 8573 (char 8572)
Expecting ',' delimiter: line 1 column 8490 (char 8489)
Expecting ',' delimiter: line 1 column 10147 (char 10146)
Expecting ',' delimiter: line 1 column 9712 (char 9711)
Expecting value: line 1 column 1 (char 0)
Expecting ',' delimiter: line 1 column 8921 (char 8920)
Expecting value: line 1 column 1 (char 0)
Expecting ',' delimiter: line 1 column 9650 (char 9649)
Expecting value: line 1 column 1 (char 0)


In [93]:
def list_of_messages_to_batch_chatgpt(questions, example_per_batch = 200, model_type = 'gpt-4o-mini', max_tokens = 4096):
    list_of_batches = []
    for i in range(0, len(questions), example_per_batch):
        
        begin_index = i
        end_index = i + example_per_batch
        print(f'Processing from {begin_index} to {end_index}')

        batch = questions[begin_index:end_index].copy()
        batch_json = []
        for j, question in enumerate(batch):
            json_obj = {
                "custom_id": question['ids'],
                "method": "POST",
                "url": '/v1/chat/completions',
                "body": {
                    "model": model_type,
                    "messages": question['message'],
                    "max_tokens":max_tokens
                },
                
            }
            batch_json.append(json_obj)
        list_of_batches.append(batch_json)
    return list_of_batches

In [6]:
# list_messages = list_of_messages_to_batch_chatgpt(questions)

In [7]:
# llm.batch_call(list_messages, transform=False)

In [ ]:
local_batch = llm.recall_local_batch()

In [ ]:
responses = llm.get_successful_messages(local_batch)

In [96]:
questions = dict()
with open('../data/generated_questions.json', 'r') as f:

    lines = json.load(f)
    for data in lines:
        ids = data['ids']
        questions[ids] = dict()
        questions[ids]['question'] = data['question']


def merge_responses(questions, responses):
    inner_join_questions = []
    for response in responses:
        questions[response['ids']]['response'] = response['response']
        inner_join_questions.append({
            'ids': response['ids'],
            'question': questions[response['ids']]['question'],
            'reasoning': response['response']
        })
    return inner_join_questions
    
inner_join_questions =merge_responses(questions, responses)

In [97]:
len(inner_join_questions)

4173

In [98]:
with open('../data/batch-response-gpt-4o-mini__v5.jsonl', 'w') as f:
    for data in inner_join_questions:
        f.write(json.dumps(data) + '\n')

In [9]:
responses

[{'ids': '5413b90f-85c4-41c2-a7c5-47324bfd44d9',
  'response': "To fulfill the request for the debt-to-equity ratio of VCB, BID, and CTG in 2023, broken down by quarter, and also compare this data to their respective industry averages, I'll create a query that fetches the following necessary information:\n\n1. **Debt-to-Equity Ratio** for each company.\n2. **Industry Average Debt-to-Equity Ratio** for 2023.\n3. Provide detailed output with relevant columns for clarity.\n\nThe steps to construct the SQL query are as follows:\n\n- First, we will select the debt-to-equity ratio (`DTER`) for VCB, BID, and CTG in the specified year and quarter.\n- Next, to find the industry averages, we'll need to aggregate the quarterly debt-to-equity ratios for all banks in 2023.\n- Finally, I'll combine both sets of data, ensuring that we are only including the relevant columns.\n\nHere’s how the SQL query is structured based on the database schema and the task requirements:\n\n```sql\n-- ### Debt-to-Equ

In [32]:
from openai import OpenAI
client = OpenAI()

client.batches.cancel("batch_67a7a453927481908a9307d1e21bfaa3")

2025-02-09 01:38:56,667 - INFO - HTTP Request: POST https://api.openai.com/v1/batches/batch_67a7a453927481908a9307d1e21bfaa3/cancel "HTTP/1.1 200 OK"


Batch(id='batch_67a7a453927481908a9307d1e21bfaa3', completion_window='24h', created_at=1739039827, endpoint='/v1/chat/completions', input_file_id='file-YTW1mToDxsUXjA5GE3xobt', object='batch', status='cancelling', cancelled_at=None, cancelling_at=1739039936, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1739126227, failed_at=None, finalizing_at=None, in_progress_at=1739039828, metadata={'description': 'batch_2'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=200))

In [27]:
for batch in client.batches.list(limit=10):
    print(batch)

2025-02-09 00:50:46,381 - INFO - HTTP Request: GET https://api.openai.com/v1/batches?limit=10 "HTTP/1.1 200 OK"


Batch(id='batch_67a794b246fc8190b36631ce32ee1601', completion_window='24h', created_at=1739035826, endpoint='/v1/chat/completions', input_file_id='file-EWrYdg6gRk9bR7sdVbWGLP', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1739036539, error_file_id=None, errors=None, expired_at=None, expires_at=1739122226, failed_at=None, finalizing_at=1739036538, in_progress_at=1739035827, metadata={'description': 'batch_0'}, output_file_id='file-Sv62sjjz9HMzCEcsYQ6dxf', request_counts=BatchRequestCounts(completed=9, failed=0, total=9))
Batch(id='batch_67a790f58a6081909155d01f8addb8e6', completion_window='24h', created_at=1739034869, endpoint='/v1/chat/completions', input_file_id='file-HAgvxerLKFxX8MW3Q1qkj8', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1739035576, error_file_id='file-XL6Lmnqy8UpqevZo9rFP8h', errors=None, expired_at=None, expires_at=1739121269, failed_at=None, finalizing_at=1739035575, in_progress_at

2025-02-09 00:50:46,749 - INFO - HTTP Request: GET https://api.openai.com/v1/batches?limit=10&after=batch_67a79044cb50819098590e0a667ade0e "HTTP/1.1 200 OK"


In [8]:
client = OpenAI()

file_response = client.files.content("file-Sv62sjjz9HMzCEcsYQ6dxf")
print(file_response.text)

2025-02-09 01:09:46,095 - INFO - HTTP Request: GET https://api.openai.com/v1/files/file-Sv62sjjz9HMzCEcsYQ6dxf/content "HTTP/1.1 200 OK"


{"id": "batch_req_67a7977a843481909962a9482a02aae7", "custom_id": "5413b90f-85c4-41c2-a7c5-47324bfd44d9", "response": {"status_code": 200, "request_id": "b6782603ceefd1c7168ff094360b254b", "body": {"id": "chatcmpl-Ayj3jSFcjIJywAGxLRB7ZjkC2t49P", "object": "chat.completion", "created": 1739036251, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "To fulfill the request for the debt-to-equity ratio of VCB, BID, and CTG in 2023, broken down by quarter, and also compare this data to their respective industry averages, I'll create a query that fetches the following necessary information:\n\n1. **Debt-to-Equity Ratio** for each company.\n2. **Industry Average Debt-to-Equity Ratio** for 2023.\n3. Provide detailed output with relevant columns for clarity.\n\nThe steps to construct the SQL query are as follows:\n\n- First, we will select the debt-to-equity ratio (`DTER`) for VCB, BID, and CTG in the specified year and quarter.\n- Next, to f

In [13]:
with open('../data/gpt-4o-mini-test.jsonl', 'w') as f:
    f.write(file_response.text)

In [14]:
data = []
with open('../data/gpt-4o-mini-test.jsonl') as f:
    for line in f:
        data.append(json.loads(line))

## Check Reasoning chain

In [2]:
import json
import re

In [109]:
data = []
with open('../data/gemini-2.0-flash-thinking-exp-01-21__v3.jsonl') as f:
    for line in f:
        try:
            data.append(json.loads(line))
        except Exception as e:
            print(e)


In [110]:
old_data = []
new_data = []

for line in data:
    if 'messages' in line and len(line['messages']) > 0:
        new_data.append(line)
    else:
        old_data.append(line)

In [111]:
len(old_data), len(new_data)

(0, 1765)

In [112]:
new_non_empty_data = []
for line in new_data:
    flag_error = False
    for msg in line['messages']:
        content = msg['content']
        role = msg['role']
        if role == 'assistant' and content == '':
            flag_error = True
            break
    if not flag_error:
        new_non_empty_data.append(line)

len(new_non_empty_data)

1740

In [113]:
new_non_empty_data[4]

{'ids': '979b5f06-a16a-4a93-80d4-14f03d2942a5',
 'date': '2025-02-12 09:43:34',
 'question': 'What was the interest coverage ratio for VietinBank and Asia Commercial Bank in 2023?',
 'table': '**Company Info**\n\n| stock_code | en_company_name | industry |\n| --- | --- | --- |\n| CTG | Vietnam Joint Stock Commercial Bank For Industry and Trade | Banking | \n| ACB | Asia Commercial Joint Stock Bank | Banking | \n\n**ratio_code_mapping**\n\n| ratio_code | ratio_name |\n| --- | --- |\n| ICR | Interest Coverage Ratio | **SQL 1 Result: Get Interest Coverage Ratio for VietinBank and Asia Commercial Bank in 2023**\n\n| stock_code | year | quarter | ratio_code | data |\n| --- | --- | --- | --- | --- |\n| ACB | 2023 | 0 | ICR | 0.26724966595740657 | \n| CTG | 2023 | 0 | ICR | 0.6865112300673836 | ',
 'reasoning': "### Decision: \nYes\n\n### Reasoning:\nThe previous SQL query successfully retrieved the Interest Coverage Ratio (ICR) for both VietinBank (CTG) and Asia Commercial Bank (ACB) for the

In [114]:
perfect_reasoning = 0
perfect_data = []
for line in new_non_empty_data:
    if len(line['messages']) == 5:
        perfect_reasoning += 1
        line['table'] = ''
        line['messages'] = line['messages']
        perfect_data.append(line)

In [115]:
perfect_reasoning

1469

In [116]:
perfect_data[345]['table']

''

In [117]:
perfect_data[345]['messages']

[{'role': 'system',
  'content': 'You are an expert in financial statement and database management. You will be asked to convert a natural language query into a SQL query. \n### Database Description\n<overall_description>\n\nThe database conatains financial statments of top Vietnamese firms, followed by the regulation of Vietnamese Accounting Standard (VAS). Based on VAS regulation, there are 3 type of financial statments: bank, corporation and securities firm, and there will be sight differences between them.\n\nThe database includes two reporting periods: quarterly and annually. Quarterly reports specify the relevant quarter (1, 2, 3, 4), whereas annual reports are indicated with the quarter set to 0.\n\n</overall_description>\n\n### PostgreSQL tables, with their properties\n```sql \n-- Table: company_info\nCREATE TABLE company_info(\n    stock_code VARCHAR(255) primary key, --The trading symbol.\n    industry VARCHAR(255), \n    exchange VARCHAR(255), -- The listed exchange of stock

In [118]:
output_path = '../data/gemini-2.0-flash-thinking-exp-01-21__hard_v0_4_good.jsonl'
with open(output_path, 'w') as f:
    for line in perfect_data:
        f.write(json.dumps(line) + '\n')

In [119]:
perfect_data[-100]

{'ids': '1aa81eff-03fc-4ca0-a1da-7617d77c7165',
 'date': '2025-02-12 18:13:32',
 'question': 'Can you provide the Gross Profit Margin for VPBank and Techcombank for the year 2021?',
 'table': '',
 'reasoning': "### Decision: \nYes\n\n### Reasoning:\nThe previous SQL query successfully retrieved the Gross Profit Margin (GPM) for both VPBank (VPB) and Techcombank (TCB) for the year 2021, as requested in the original task. The SQL result table confirms that data for 'GPM' is available for both stock codes ('TCB', 'VPB') in the year 2021 and quarter 0 (annual report). Thus, the query is correct and provides the answer to the user's request.",
 'sql': [],
 'messages': [{'role': 'system',
   'content': 'You are an expert in financial statement and database management. You will be asked to convert a natural language query into a SQL query. \n### Database Description\n<overall_description>\n\nThe database conatains financial statments of top Vietnamese firms, followed by the regulation of Viet

In [120]:
correct_reasoning = 0
correct_data = []
not_perfect_noNo = []
yes_no_data = []

for line in new_non_empty_data:
    is_no = False
    is_yes = False
    for ms in line['messages']:
        if '###decision:\nyes' in ms['content'].lower().replace(' ', ''):
            is_yes = True
        if '###decision:\nno' in ms['content'].lower().replace(' ', ''):
            is_no = True
    if not is_no and is_yes:
        correct_reasoning += 1
        correct_data.append(line)
        
        if len(line['messages']) != 5:
            not_perfect_noNo.append(line)
    if is_no and is_yes:
        yes_no_data.append(line)

        correct_reasoning += 1

correct_reasoning

1625

In [121]:
len(yes_no_data)

122

In [122]:
len(not_perfect_noNo)

34

In [123]:
not_perfect_noNo[20]

{'ids': 'f909eb41-17c9-4041-9a0c-fdc065e7db50',
 'date': '2025-02-12 14:26:59',
 'question': 'What is the Debt-to-Equity ratio of BIDV for the year 2022 compared to the industry average?',
 'table': '**Company Info**\n\n| stock_code | en_company_name | industry |\n| --- | --- | --- |\n| BID | Joint Stock Commercial Bank for Investment and Development of Vietnam | Banking | **ratio_code_mapping**\n\n| ratio_code | ratio_name |\n| --- | --- |\n| DTER | Debt To Equity Ratio | **Debug Round 1 SQL 1 Result: financial_ratio**\n\n| stock_code | year | quarter | ratio_code | data |\n| --- | --- | --- | --- | --- |\n| BIDV | 2022 | 0 | DTER | 19.367913571175464 | ',
 'reasoning': "### Decision: \nYes\n\n### Reasoning:\nThe previous SQL query was almost correct, but it had a syntax error due to a semicolon placed before `UNION ALL`.  The semicolon prematurely terminated the first `SELECT` statement, causing the SQL parser to not recognize `UNION ALL` in the expected context. Removing the semicol

In [124]:
yes_no_data[50]

{'ids': '331a5526-88a5-4cca-bd40-caa0e6cc20b1',
 'date': '2025-02-12 13:43:31',
 'question': "Please share the accounts payable turnover ratio for Eximbank in 2021, and discuss its significance in assessing the bank's payment efficiency.",
 'table': '**Company Info**\n\n| stock_code | en_company_name | industry |\n| --- | --- | --- |\n| EIB | Vietnam Export Import Commercial Joint Stock Bank | Banking | **ratio_code_mapping**\n\n| ratio_code | ratio_name |\n| --- | --- |\n| DPO | Days Payable Outstanding | | RTPR | Receivables To Payables Ratio | \n**SQL 1 Result: financial_ratio**\n\n| stock_code | year | quarter | ratio_code | data |\n| --- | --- | --- | --- | --- |\n**Correction SQL 1 Result: financial_ratio**\n\n| stock_code | year | quarter | ratio_code | data |\n| --- | --- | --- | --- | --- |\n| EIB | 2021 | 0 | RTPR | 0.00918871163477393 | ',
 'reasoning': '### Decision: \nYes\n\n### Reasoning:\nThe corrected SQL query successfully retrieved the \'RTPR\' (Receivables To Payable

In [125]:
usable_yes_no = []
for line in yes_no_data:
    flag_error = False
    for i, msg in enumerate(line['messages']):
        content = msg['content']
        role = msg['role']
        if role == 'user' and 'Error Messages from previous query' in content:
            if '###decision:\nno' in line['messages'][i+1]['content'].lower().replace(' ', ''):
                flag_error = True
                break
    if not flag_error:
        usable_yes_no.append(line)

In [126]:
len(usable_yes_no)

120

In [127]:
len(not_perfect_noNo)

34

In [128]:
usable_not_perfect_noNo = []
for line in not_perfect_noNo:
    flag_error = False
    for msg in line['messages']:
        content = msg['content']
        role = msg['role']
        if role == 'user' and 'Error Messages from previous query' in content:
            flag_error = True
            break
    if not flag_error:
        usable_not_perfect_noNo.append(line)

In [129]:
len(usable_not_perfect_noNo)

14

In [130]:
re_thinking_data = usable_yes_no + usable_not_perfect_noNo

In [131]:
import sys 
sys.path.append('../')
from llm import Gemini, RotateGemini

llm = RotateGemini(rpm=14)

def summarize_reasoning_chain(llm, messages):
    summarize_prompt = "Summarize and Return the correct reasoning chain and the SQL query, under ### Reasoning: and ### SQL Query: respectively."

    messages.append({
        "role": "user",
        "content": summarize_prompt
    })

    response = llm(messages)

    return response

Found 4 API keys


In [132]:
re_thinking_data[0]['messages']

[{'role': 'system',
  'content': "You are an expert in financial statement and database management. You will be asked to convert a natural language query into a SQL query. \n### Database Description\nYou're given database about the financial statments of top Vietnamese companies, either bank, cooperates and securities.\n\n<overall_description>\n\nAll the data in the financial statments are followed by the regulation of Vietnamese Accounting Standard (VAS). The English translations are followed by the International Financial Reporting Standards (IFRS).\n\nThere are 3 type of financial statments, based on VAS regulation: bank, non-bank corporation and securities firm (firms that provide stock options and financial instruments).\nAll 3 type of reports are stored in one single table, and there will be a sight different between them.\n\nThe database includes two reporting periods for financial statements: quarterly and annually. Quarterly reports specify the relevant quarter (1, 2, 3, 4), w

In [89]:
test = summarize_reasoning_chain(llm, re_thinking_data[0]['messages'])

2025-02-14 14:19:10,511 - INFO - AFC is enabled with max remote calls: 10.


gemini-2.0-flash


2025-02-14 14:19:16,704 - INFO - AFC remote call 1 is done.
2025-02-14 14:19:16,705 - INFO - Completion time of gemini-2.0-flash: 6.193599462509155s


cached_content_token_count=None candidates_token_count=874 prompt_token_count=13210 total_token_count=14084


In [90]:
test

"### Reasoning:\nThe user wants to retrieve and compare the Debt-to-Equity ratio (DTER) for specific Real Estate companies against the industry average. The data should be annual for the years 2020 to 2023 and for Q3 of 2024. We need to query the `financial_ratio` table for company-specific data and the `industry_financial_ratio` table for industry average data. Then, we will combine these results for comparison.\n\n**Steps:**\n\n1.  **Retrieve Company-Specific DTER:** Select `stock_code`, `year`, `quarter`, and `data` from the `financial_ratio` table where `ratio_code` is 'DTER' and `stock_code` is in the provided list of real estate companies (SZC, VHM, VRE, KBC, NVL, DXG, SCR, KDH, HDC, TCH, CEO, VC3, TIG, HLD, DTD). Filter for years 2020 to 2023 (annual reports, quarter=0) and Q3 2024 (quarter=3).\n2.  **Retrieve Industry Average DTER:** Select `industry`, `year`, `quarter`, and `data_mean` from the `industry_financial_ratio` table where `ratio_code` is 'DTER' and `industry` is 'Re

In [ ]:
clean_reasoning = []
for data in re_thinking_data:
    data['table'] = ''
    messages = data['messages'].copy()
    summarize = summarize_reasoning_chain(llm, messages)

    messages = messages[:2]
    messages.append({
        "role": "assistant",
        "content": summarize
    })
    data2 = data.copy()
    data2['messages'] = messages

    clean_reasoning.append(data2)   
    print(summarize)


In [92]:
clean_reasoning[0]


{'ids': '586f7627-5929-4ad2-ab00-45dee24e0fdf',
 'date': '2025-02-14 00:34:15',
 'question': "Calculate the Debt-to-Equity ratio for each of the Real Estate companies (SZC, VHM, VRE, KBC, NVL, DXG, SCR, KDH, HDC, TCH, CEO, VC3, TIG, HLD, DTD) annually from 2020 to Q3 2024.  Compare each company's ratio to the industry average for each year.",
 'table': '',
 'reasoning': "### Decision: \nYES\n\n### Reasoning:\nThe corrected SQL query successfully retrieves the Debt-to-Equity ratio for each specified Real Estate company and the industry average for each year from 2020 to 2023 and Q3 2024. The result table combines both company-specific DTER and industry average DTER in the same row, making it easy to compare each company's ratio to the industry average for the corresponding year and quarter, as requested in the original task. The table structure is suitable for the task.",
 'sql': [],
 'messages': [{'role': 'system',
   'content': "You are an expert in financial statement and database ma

In [93]:
# select_data = perfect_data + usable_yes_no + usable_not_perfect_noNo

select_data = perfect_data + clean_reasoning

In [94]:
len(select_data)


1656

In [95]:
select_data[0]

{'ids': '67644a21-6c8d-4f43-99cd-e23e5f6bb60b',
 'date': '2025-02-14 00:27:30',
 'question': 'What were the total revenues of Vinamilk and Masan Group in 2022?',
 'table': '',
 'reasoning': "### Decision: \nYES\n\n### Reasoning:\nThe SQL query successfully retrieved the total revenues for Vinamilk (VNM) and Masan Group (MSN) in 2022, as requested in the original task. The result table clearly displays the stock codes, year, quarter, category code (Revenue from sales and services rendered), and the corresponding revenue data for both companies. This directly answers the user's question, and the table format is appropriate for presenting this information.",
 'sql': [],
 'messages': [{'role': 'system',
   'content': "You are an expert in financial statement and database management. You will be asked to convert a natural language query into a SQL query. \n### Database Description\nYou're given database about the financial statments of top Vietnamese companies, either bank, cooperates and s

1427 Perfect Query
1695 Correct Query

### Correct Query with one Correction and msg > 5 (fixing bug perhaps)

In [52]:
def print_messages(messages):
    for msg in messages:
        print("===========",msg['role'],"============")
        print(msg['content'])

In [ ]:
count = 0
for line in correct_data:
    
    if len(line['messages']) != 5:
        count += 1
        if count == 5:
            print_messages(line['messages'])
            break

In [14]:
data[100]

{'ids': '34e71951-99ae-4693-8b7b-e00683d3b17b',
 'question': "For the period 2020-Q3 2024, calculate the equity multiplier (Total Assets / Total Equity) for each company in the Healthcare industry. Compare each company's trend to the industry average. What does this reveal about the use of financial leverage within the industry and how this has changed over time?",
 'table': '**SQL 1 Result: quarterly_equity_multiplier_healthcare_industry**\n\n| stock_code | industry | year | quarter | company_equity_multiplier | avg_industry_equity_multiplier |\n| --- | --- | --- | --- | --- | --- |\n| DVM | Healthcare | 2022 | 1 | 2.454875996056095 | 2.454875996056095 |\n| DVM | Healthcare | 2022 | 2 | 2.056330834703408 | 2.056330834703408 |\n| DVM | Healthcare | 2022 | 3 | 2.045280159369722 | 2.045280159369722 |\n| DVM | Healthcare | 2022 | 4 | 2.0927981883630897 | 2.0927981883630897 |\n| DVM | Healthcare | 2023 | 1 | 2.1096495790338303 | 2.1096495790338303 |\n| DVM | Healthcare | 2023 | 2 | 2.15234

## Rerun SQL -> get table

In [1]:
import sys 
sys.path.append('../')
from initialize import initialize_text2sql

from agent.prompt.prompt_controller import (
    FIIN_VERTICAL_PROMPT_UNIVERSAL,
    FIIN_VERTICAL_PROMPT_UNIVERSAL_SIMPLIFY,
    FIIN_VERTICAL_PROMPT_UNIVERSAL_OPENAI,
    FIIN_VERTICAL_PROMPT_UNIVERSAL_SHORT
)

from agent.const import (
    TEXT2SQL_FAST_OPENAI_CONFIG,
)
from agent.text2sql_utils import reconstruct_tables_from_sql, table_to_markdown
from llm.llm_utils import get_code_from_text_response

solver = initialize_text2sql(TEXT2SQL_FAST_OPENAI_CONFIG, FIIN_VERTICAL_PROMPT_UNIVERSAL_OPENAI)

2025-02-16 22:40:08,019 - INFO - Checking embedding server at http://localhost:8080/embed
2025-02-16 22:40:08,024 - INFO - Response embedding server: <Response [200]>
2025-02-16 22:40:08,024 - INFO - Using remote embedding server
2025-02-16 22:40:08,024 - INFO - Finish setup embedding
2025-02-16 22:40:08,024 - INFO - Checking embedding server at http://localhost:8081
2025-02-16 22:40:08,031 - INFO - Response embedding server: <Response [200]>
2025-02-16 22:40:08,032 - INFO - Found reranking server at: http://localhost:8081
2025-02-16 22:40:08,032 - INFO - Finish setup reranker, using reranker api
2025-02-16 22:40:08,036 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-02-16 22:40:08,105 - INFO - Finish setup db
2025-02-16 22:40:08,105 - INFO - Using ChatGPT with model gpt-4o-mini
2025-02-16 22:40:08,119 - INFO - Using ChatGPT with model gpt-4o-mini
2025-02-16 22:40:08,133 - INFO - Finish setup text2sql


fiin


In [2]:
import json

In [3]:
input_path = '../data/gemini-2.0-flash-thinking-exp-01-21__v1_good.jsonl'
questions = []

with open(input_path) as f:
    for line in f:
        data = json.loads(line)
        questions.append(data)

In [ ]:
runnable_questions = []

for question in questions:
    code = get_code_from_text_response(question['messages'][-1]['content'])
    if code[0]['language'] == 'sql':
        # print('========== Code ===========')
        # print(code[0]['code'])
        tables = reconstruct_tables_from_sql(solver.db, question['messages'][-1]['content'])
        if len(tables) > 0:
            tables = table_to_markdown(tables)
            question['table'] = tables
            runnable_questions.append(question)

len(runnable_questions)


In [6]:
output_path = '../data/gemini-2.0-flash-thinking-exp-01-21__v1_good_runnable.jsonl'
with open(output_path, 'w') as f:
    for line in runnable_questions:
        f.write(json.dumps(line) + '\n')

## Batch

In [3]:
template_path = '../data/template_gemini-2.0-flash-lite-preview-02-05__v5.jsonl'
batch_path = '../data/batch-response-gpt-4o-mini__v5.jsonl'
output_path = '../data/gpt-4o-mini__v5_2.jsonl'

In [14]:
template_data = []
with open(template_path) as f:
    for line in f:
        try:
            template_data.append(json.loads(line))
            
        except Exception as e:
            print(e)

reponses = dict()
questions = dict()
with open(batch_path) as f:
    for line in f:
        try:
            data = json.loads(line)
            reponses[data['ids']] = data['reasoning']
            questions[data['ids']] = data['question']
        except Exception as e:
            print(e)

Expecting ',' delimiter: line 1 column 8573 (char 8572)
Expecting ',' delimiter: line 1 column 8490 (char 8489)
Expecting ',' delimiter: line 1 column 10147 (char 10146)
Expecting ',' delimiter: line 1 column 9712 (char 9711)
Expecting value: line 1 column 1 (char 0)
Expecting ',' delimiter: line 1 column 8921 (char 8920)
Expecting value: line 1 column 1 (char 0)
Expecting ',' delimiter: line 1 column 9650 (char 9649)
Expecting value: line 1 column 1 (char 0)


In [5]:
template_data[0]

{'date': '2025-08-02 23:53:58',
 'ids': '5413b90f-85c4-41c2-a7c5-47324bfd44d9',
 'message': [{'role': 'system',
   'content': "You are an expert in financial statement and database management. You will be asked to convert a natural language query into a SQL query. \nYou will have the following database description:\n\n### Database Description\nYou're given database about the financial statments of top Vietnamese companies, either bank, cooperates and securities.\n\n<overall_description>\n\nAll the data in the financial statments are followed by the regulation of Vietnamese Accounting Standard (VAS). The English translations are followed by the International Financial Reporting Standards (IFRS).\n\nThere are 3 type of financial statments, based on VAS regulation: bank, non-bank corporation and securities firm (firms that provide stock options and financial instruments).\nAll 3 type of reports are stored in one single table, and there will be a sight different between them.\n\nThe databa

In [5]:
len(template_data)

4173

In [6]:
import logging
from tqdm import tqdm

logging.disable(logging.CRITICAL)

In [ ]:
selected_data = []
for line in tqdm(template_data):
    ids = line['ids']
    if ids in reponses:
        line['question'] = questions[ids]
        line['reasoning'] = reponses[ids]
        line['message'].append(
            {
                "role": "assistant",
                "content": reponses[ids]
            }
        )
        tables = reconstruct_tables_from_sql(solver.db, reponses[ids])
        if len(tables) > 0:
            tables = table_to_markdown(tables, adjust='text')
            line['table'] = tables
            selected_data.append(line)


In [15]:
len(selected_data)

3575

In [18]:
selected_data[0]

{'date': '2025-08-02 23:53:58',
 'ids': '5413b90f-85c4-41c2-a7c5-47324bfd44d9',
 'message': [{'role': 'system',
   'content': "You are an expert in financial statement and database management. You will be asked to convert a natural language query into a SQL query. \nYou will have the following database description:\n\n### Database Description\nYou're given database about the financial statments of top Vietnamese companies, either bank, cooperates and securities.\n\n<overall_description>\n\nAll the data in the financial statments are followed by the regulation of Vietnamese Accounting Standard (VAS). The English translations are followed by the International Financial Reporting Standards (IFRS).\n\nThere are 3 type of financial statments, based on VAS regulation: bank, non-bank corporation and securities firm (firms that provide stock options and financial instruments).\nAll 3 type of reports are stored in one single table, and there will be a sight different between them.\n\nThe databa

In [16]:
for data in selected_data:
    ids = data['ids']
    data['question'] = questions[ids]

In [17]:
with open(output_path, 'w') as f:
    for line in selected_data:
        f.write(json.dumps(line) + '\n')

In [8]:
text = """ 
{a}
 smt
 {b}
"""

text.format(a='a', b='b', c= 'c')

' \na\n smt\n b\n'

## Change system prompt

In [2]:
import random

def get_random_sys_prompt():
    choices = {
        'vertical': FIIN_VERTICAL_PROMPT_UNIVERSAL['OPENAI_SEEK_DATABASE_PROMPT'],
        'openai': FIIN_VERTICAL_PROMPT_UNIVERSAL_OPENAI['OPENAI_SEEK_DATABASE_PROMPT'],
        'short': FIIN_VERTICAL_PROMPT_UNIVERSAL_SHORT['OPENAI_SEEK_DATABASE_PROMPT'],
        'simpify': FIIN_VERTICAL_PROMPT_UNIVERSAL_SIMPLIFY['OPENAI_SEEK_DATABASE_PROMPT']

    }

    choice = random.choice(['openai','openai','openai', 'short', 'simpify', 'simpify'])

    prompt = """ 
You are an expert in financial statement and database management. You will be asked to convert a natural language query into a SQL query. 
### Database Description
{database}
"""
    
    return prompt.format(database=choices[choice])



In [133]:
output_path = '../data/gemini-2.0-flash-thinking-exp-01-21__v3.jsonl'
with open(output_path, 'w') as f:
    for line in perfect_data:
        line['messages'][0]['content'] = get_random_sys_prompt()
        f.write(json.dumps(line) + '\n')

In [55]:
msg_paths = [
    '../data/message_gpt-4o-mini__v4_good.jsonl',
    '../data/message_gpt-4o-mini__v5_2_good.jsonl',
    '../data/message_gpt-4o-mini__v5_good.jsonl',
]
output_path = '../data/message_gpt-4o-mini.jsonl'

count_msg = 0

for msg_path in msg_paths:
    with open(msg_path) as f:
        for line in f:
            try:
                messages = json.loads(line)
                if messages[0].get('role') == 'system':
                    messages[0]['content'] = get_random_sys_prompt()

                with open(output_path, 'a') as f:
                    f.write(json.dumps(messages) + '\n')
                count_msg += 1
            except Exception as e:
                print(e)

count_msg


Expecting ',' delimiter: line 1 column 12128 (char 12127)
Expecting value: line 1 column 1 (char 0)
Expecting ',' delimiter: line 1 column 12017 (char 12016)
Expecting value: line 1 column 1 (char 0)
Expecting ',' delimiter: line 1 column 11098 (char 11097)
Expecting value: line 1 column 1 (char 0)
Expecting ',' delimiter: line 1 column 8201 (char 8200)
Expecting value: line 1 column 1 (char 0)
Expecting ',' delimiter: line 1 column 8820 (char 8819)
Expecting value: line 1 column 1 (char 0)
Expecting ',' delimiter: line 1 column 11078 (char 11077)
Expecting value: line 1 column 1 (char 0)


4883

In [82]:
output_path = '../data/message_correction__v1.jsonl'
with open(output_path, 'w') as f:
    for line in perfect_data:
        msg = line['messages']
        system_prompt = get_random_sys_prompt()
        system_prompt += """ 
### Regulation:   
- The result of the SQL query will be displayed in the <data> tag.
- Your should reasoning and provide the SQL query with explaination for the task in <task> and <instruction> tag.
- For <correction> tag, you only need to provide the SQL query if the previous SQL query is incorrect.
- For <reflection> tag, you dont need to provide the SQL query."""
        msg[0]['content'] = system_prompt
        choice = random.choice([0,1,2,3])
        if choice < 3:
            msg = msg[:3]
        
        f.write(json.dumps(msg) + '\n')

In [83]:
long_query = usable_yes_no + usable_not_perfect_noNo

with open(output_path, 'a') as f:
    for line in long_query:
        msg = line['messages']
        system_prompt = get_random_sys_prompt()
        system_prompt += """ 
### Regulation:   
- The result of the SQL query will be displayed in the <data> tag.
- Your should reasoning and provide the SQL query with explaination for the task in <task> and <instruction> tag.
- For <correction> tag, you only need to provide the SQL query if the previous SQL query is incorrect.
- For <reflection> tag, you dont need to provide the SQL query."""
        msg[0]['content'] = system_prompt
        f.write(json.dumps(msg) + '\n')

In [1]:
import re

def clean_text(text):
    pattern = r'\s\([A-Z]{3,}\)'
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

# Example usage
text = "How did the return on equity (ROE) of Asia Commercial Bank (ACB) compare to TPBank (TPB) in 2021?"
cleaned_text = clean_text(text)
print(cleaned_text)


How did the return on equity of Asia Commercial Bank compare to TPBank in 2021?


In [6]:
import json 
import random

hard_path = '../data/hard_questions_v0_4.jsonl'
hard_data = []
with open(hard_path) as f:
    for line in f:
        data = json.loads(line)
        choice = random.choice([0,1,2,3, 4 ])

        if choice < 4:
            data['question'] = clean_text(data['question'])
        hard_data.append(data)

output_path = '../data/hard_questions_v0_4.jsonl'
with open(output_path, 'w') as f:
    for line in hard_data:
        line2 = dict()
        line2['question'] = line['question']
        line2['ids'] = line['ids']
        f.write(json.dumps(line2) + '\n')

In [6]:
import sys
sys.path.append('../')
from llm.llm_utils import get_json_from_text_response

In [9]:
input_path = '../data/ner_gemini-2.0-flash__v5.jsonl'
import random
import json

# 1/4 keep

output_path = '../data/message_ner_gemini-2.0-flash__v5.jsonl'
count = 0
with open(input_path) as f:
    for line in f:
        ner_msg = json.loads(line)['ner']
        if random.randint(0, 3) != 0:

            for msg in ner_msg:
                msg['content'] = clean_text(msg['content'])

        flag_pass = True  
        for msg in ner_msg:
            if msg['role'] == 'assistant':
                try:
                    get_json_from_text_response(msg['content'])
                except Exception as e:
                    print(e)
                    flag_pass = False
                    print(msg['content'])
                    break
        if flag_pass:
            with open(output_path, 'a') as f:
                f.write(json.dumps(ner_msg) + '\n')
            count += 1
        if count == 2000:
            break
    

No JSON response found in text response
No JSON response found in text response


## Get DPO pair

In [3]:
## Gemini Thinking
import json
choosen_data = dict()

path_1 = '../data/gemini-2.0-flash-thinking-exp-01-21__v3.jsonl'
path_2 = '../data/gemini-2.0-flash-thinking-exp-01-21__hard_v0_4.jsonl'

In [4]:
with open(path_1) as f:
    for line in f:
        data = json.loads(line)
        choosen_data[data['ids']] = data

with open(path_2) as f:
    for line in f:
        data = json.loads(line)
        choosen_data[data['ids']] = data

In [5]:
rejected_data = dict()


def get_rejected_data(data_path, good_path):

    good_ids = set()
    rejected_data = dict()

    with open(good_path) as f:
        for line in f:
            data = json.loads(line)
            good_ids.add(data['ids'])
    
    with open(data_path) as f:
        for line in f:
            data = json.loads(line)
            if data['ids'] not in good_ids:
                rejected_data[data['ids']] = data
    return rejected_data


data_path_1 = '../data/deepseek-chat__v0.jsonl'
good_path_1 = '../data/deepseek-chat__v0_good.jsonl'

data_path_2 = '../data/qwen2.5-coder-3b-sft__v3.jsonl'
good_path_2 = '../data/qwen2.5-coder-3b-sft__v3_good.jsonl'

data_path_3 = '../data/qwen2.5-coder-3b-sft__hard_v0_4.jsonl'
good_path_3 = '../data/qwen2.5-coder-3b-sft__hard_v0_4_good.jsonl'

data_path_4 = '../data/gpt-4o-mini__v3.jsonl'
good_path_4 = '../data/gpt-4o-mini__v3_good.jsonl'

rejected_data_1 = get_rejected_data(data_path_1, good_path_1)
rejected_data_2 = get_rejected_data(data_path_2, good_path_2)
rejected_data_3 = get_rejected_data(data_path_3, good_path_3)
rejected_data_4 = get_rejected_data(data_path_4, good_path_4)



for ids, data in rejected_data_2.items():
    if ids not in rejected_data:
        rejected_data[ids] = []
    rejected_data[ids].append(data)

for ids, data in rejected_data_3.items():
    if ids not in rejected_data:
        rejected_data[ids] = []
    rejected_data[ids].append(data)

for ids, data in rejected_data_4.items():
    if ids not in rejected_data:
        continue
        rejected_data[ids] = []
    
    ref_data = rejected_data[ids]

    data['messages'] = ref_data[0]['messages'][:2]
    data['messages'].append(
        {
            "role": "assistant",
            "content": data['reasoning']
        }
    )
    rejected_data[ids].append(data)
    

for ids, data in rejected_data_1.items():
    if ids not in rejected_data:
        continue
        rejected_data[ids] = []
    
    ref_data = rejected_data[ids]

    data['messages'] = ref_data[0]['messages'][:2]
    data['messages'].append(
        {
            "role": "assistant",
            "content": data['reasoning']
        }
    )
    rejected_data[ids].append(data)

In [6]:
len(rejected_data)

1725

## SFT

In [27]:
sft_data = []
for ids, data in choosen_data.items():
    # if ids in rejected_data:
    choice = random.choice([0,1])

    if choice == 0:
         if data['ids'] not in rejected_data:
            continue

    
    messages = data['messages']
    messages[0]['content'] = get_random_sys_prompt().strip()
    messages[0]['content'] += """
### Regulation:   
- The result of the SQL query will be displayed in the <data> tag.
- Your should reasoning and provide the SQL query with explaination for the task in <task> and <instruction> tag.
- For <correction> tag, you only need to provide the SQL query if the previous SQL query is incorrect.
- For <reflection> tag, you dont need to provide the SQL query."""

    if len(messages) == 5:
            choice = random.choice([0,1])
            if choice == 0:
                messages = messages[:3]

    sft_data.append({
        'messages': messages,
    })


In [28]:
len(sft_data)

2212

In [29]:
sft_data[0]

{'messages': [{'role': 'system',
   'content': 'You are an expert in financial statement and database management. You will be asked to convert a natural language query into a SQL query. \n### Database Description\n<overall_description>\n\nThe database conatains financial statments of top Vietnamese firms, followed by the regulation of Vietnamese Accounting Standard (VAS). Based on VAS regulation, there are 3 type of financial statments: bank, corporation and securities firm, and there will be sight differences between them.\n\nThe database includes two reporting periods: quarterly and annually. Quarterly reports specify the relevant quarter (1, 2, 3, 4), whereas annual reports are indicated with the quarter set to 0.\n\n</overall_description>\n\n### PostgreSQL tables, with their properties, in OpenAI Template\n\n<schema>\n\n- **company_info**(stock_code, industry, exchange, stock_indices, is_bank, is_securities)\n- **sub_and_shareholder**(stock_code, invest_on)\n- **financial_statement

## KTO

In [8]:
pair_able  = 0

absolute_true = []
mixed_true = []
mixed_false = []
absolute_false = []

for ids, data in choosen_data.items():
    if ids in rejected_data:
        pair_able += 1
        mixed_true.append(data)
        for value in rejected_data[ids]:
            mixed_false.append(value)
    else:
        absolute_true.append(data)

pair_able



1325

In [23]:
len(absolute_true), len(mixed_true), len(mixed_false)

(1800, 1325, 1808)

In [24]:
import random

ran_abs_true = random.sample(absolute_true, 800)
ran_mixed_true = random.sample(mixed_true, 1000)
ran_mixed_false = random.sample(mixed_false, 1800)

In [32]:
kto_data = []

In [33]:
for data in ran_abs_true:
    ids = data['ids']
    message = data['messages'].copy()[:3]
    message[0]['content'] = get_random_sys_prompt().strip()

    kto_data.append({
        'label': True,
        'messages': message
    })

for data in ran_mixed_true:
    ids = data['ids']
    message = data['messages'].copy()[:3]
    message[0]['content'] = get_random_sys_prompt()

    kto_data.append({
        'label': True,
        'messages': message
    })

for data in ran_mixed_false:
    ids = data['ids']
    message = data['messages'].copy()[:3]
    message[0]['content'] = get_random_sys_prompt()

    kto_data.append({
        'label': False,
        'messages': message
    })

In [34]:
len(kto_data)

3600

In [35]:
kto_data = random.sample(kto_data, len(kto_data))

In [36]:
output_path = '../data/message_kto.json'
with open(output_path, 'w') as f:
    json.dump(kto_data, f, indent=4)

## DPO

In [37]:
import random

dpo_data = []

for ids, data in choosen_data.items():
    question = data['question']
    ids = data['ids']
    message = data['messages'].copy()
    message[0]['content'] = get_random_sys_prompt()
    choosen = message[2]
    message = message[:2]

    if ids in rejected_data:
        num_sample = random.randint(1, len(rejected_data[ids]))
        for i in range(num_sample):

            if 'messages' in rejected_data[ids][i]:
                rejected = rejected_data[ids][i]['messages'][2]
            else:
                rejected = {
                    'role': 'assistant',
                    'content': rejected_data[ids][i]['reasoning']
                }
            if rejected['content'] == '':
                continue

            dpo_data.append({
                'question': question,
                'choosen' : choosen,
                'messages': message,
                'rejected': rejected,
                'ids': ids
            })

In [38]:
len(dpo_data)

2064

In [39]:
with open('../data/dpo_data.json', 'w') as f:
    json.dump(dpo_data, f, indent=4)

In [34]:
import tiktoken

def count_tokens(text: str, model: str = "gpt-4") -> int:
    # Select the tokenizer for the model
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

total_tokens = 0
max_tokens = 0
moreThan2000 = 0
moreThan3000 = 0
moreThan4000 = 0

for data in sft_data:
    local_tokens = 0
    for msg in data['messages']:
        local_tokens += count_tokens(msg['content'])

    # accepted_tokens = count_tokens(data['choosen']['content'])
    # reject_token = count_tokens(data['rejected']['content'])

    # max_local_tokens = max(local_tokens + accepted_tokens, accepted_tokens + reject_token)
    # max_tokens = max(max_tokens, max_local_tokens)

    # total_tokens += local_tokens + accepted_tokens + reject_token
    max_tokens = max(max_tokens, local_tokens)
    max_local_tokens = local_tokens

    total_tokens += local_tokens

    if max_local_tokens > 2000:
        moreThan2000 += 1
    if max_local_tokens > 3000:
        moreThan3000 += 1
    if max_local_tokens > 6000:
        moreThan4000 += 1

total_tokens, max_tokens, moreThan3000, moreThan4000    

(6053209, 7394, 545, 18)

In [35]:
output_path = '../data/message_sft.json'
with open(output_path, 'w') as f:
    json.dump(sft_data, f, indent=4)

In [33]:
moreThan2000

1951